<a href="https://colab.research.google.com/github/echughes529/quantum_network_simulation/blob/main/modelling_entanglement_generation_more_rigorously.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qutip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 16.2 MB/s eta 0:00:00


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *


## Setting up

In [5]:
# setting up state
zero_ket = basis(2,0)
zero_rho = basis(2,0) * basis(2,0).dag()
num_qubits = 4
psi0 = tensor([zero_ket] * num_qubits)


# defining gates
H = lambda: (1/np.sqrt(2)) * (sigmaz() + sigmax())
CNOT = lambda: Qobj([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]], dims=[[2, 2], [2, 2]])
I = lambda: qeye(2)


## Entanglement Generation

In [6]:
def entanglement_generation_event(q1, q2):
  num_qubits_left = q1
  num_qubits_right = (num_qubits - 1) - q2

  # entanglement op over the 2 qubits
  entanglement_op = CNOT() * tensor(H(), I())

  # generate identity gates to the left (if there are any) and adds to the entanglment op
  if num_qubits_left > 0:
    left_op = I()
    for i in range(num_qubits_left-1):
      left_op = tensor(left_op, I())
    entanglement_op = tensor(left_op, entanglement_op)

  # generate identity gates to the right (if there are any) and adds to the entanglment op
  if num_qubits_right>0:
    right_op = I()
    for i in range(num_qubits_right-1):
      right_op = tensor(right_op, I())
    entanglement_op = tensor(entanglement_op, right_op)

  return entanglement_op

In [7]:
# testing function
def test_entang_gen_w_statevec(qubit1, qubit2):
  """
  Testing the entanglment generation function with statevectors
  """
  psi0 = tensor([zero_ket] * num_qubits)
  for i in range (num_qubits):
    print(f'Repeater {i}\n', (entanglement_generation_event(qubit1, qubit2) * psi0).ptrace(i).full(), '\n')

test_entang_gen_w_statevec(0, 1)

Repeater 0
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 

Repeater 1
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 

Repeater 2
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Repeater 3
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 



Entanglement preparation time: $T_p$

Time to travel from one repeater to another: $\frac{d}{c}$

Total time to establish entanglement link: $T_p + 2\frac{d}{c}$

Channel efficiency: $\eta_{ch}(L) = e^{-\frac{L}{L_{att}}}$

Total probability that a pair is established between two neighbouring repeaters separated by a distance $L$ in one trial: $\eta = P_{link} \times \eta_{ch}(L)$

In [10]:
# modelling time it takes to complete entangelment generation event
L_att = 22e3  # attenuation length of optical fibers, 22km

# prob pair is established not taking into account distance based losses
P_link = 1

# channel efficiency
n_ch = lambda L: np.exp(-L/L_att)

# total probability that a pair is established
#n = P_link * n_ch


## Dark Counts

The probability that a dark count will occur in a detection window: $p_d$. For a detector with a dark count rate of 1 Hz and detection window of $1 \mu s$ we get $p_d = 10^{-6}$

In [9]:
T_p = 1e-3
num_qubits = 5
total_L = 100
d = total_L/(num_qubits - 1)
c = 2e8 # speed of light in fiber optic
eta_eff = 0.5  # Success probability


def time_for_link(eta_eff):
    """
    Gets the time taken to generate entanglement between 2 adjacent nodes
    Does this by generating random samples from a geometric distribution with
    success probability eta_eff and then multiplying by time taken for one trial.

    Parameters:
    eta_eff (float): Success probability for the geometric distribution.
    num_samples (int): Number of random samples to generate.

    Returns:
    np.ndarray: Array of random samples from the geometric distribution.
    """
    no_of_trials = np.random.geometric(eta_eff)
    time_per_trial = T_p + 2 * d / c
    time_per_link = no_of_trials * time_per_trial
    return time_per_link


samples = time_for_link(eta_eff)
print(samples)


0.0020005


# Modelling Decoherence Error

When generating entanglement we always generate the entangled pair in the lowest station and then send one of these entangled qubits to the higher station.

This means that q0 experiences memory noise for 2d/c and q1 experiences memory noise for d/c in the entanglement event

In [14]:
def bell_state(state):
  """
    returns requested bell state
  """
  if state == 'phi+':
      phi_plus_ket = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_plus_ket * phi_plus_ket.dag()

  elif state == 'phi-':
      phi_minus_ket = (tensor(basis(2, 0), basis(2, 0)) - tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_minus_ket * phi_minus_ket.dag()

  elif state == 'psi+':
      psi_plus_ket = (tensor(basis(2, 0), basis(2, 1)) + tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_plus_ket * psi_plus_ket.dag()

  elif state == 'psi-':
      psi_minus_ket = (tensor(basis(2, 0), basis(2, 1)) - tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_minus_ket * psi_minus_ket.dag()

  else:
      raise ValueError("Invalid Bell state label")


def pad_op_to_left(op, num_identities):
  if num_identities > 0:
    left_op = I()
    for i in range(num_identities - 1):
      left_op = tensor(left_op, I())
    return tensor(left_op, op)
  else:
    return op

def pad_op_to_right(op, num_identities):
  if num_identities > 0:
    right_op = I()
    for i in range(num_identities - 1):
      right_op = tensor(right_op, I())
    return tensor(op, right_op)
  else:
    return op




In [11]:
def entanglement_generation_with_ptracing(rho_initial, q1, q2, F_initial):
  """
    This function generates entanglement across 2 neighbouring repeater stations
    It does this by first creating a bell pair across 2 qubits, padding this out
    with identities so that it matches the size of the network, then it multiplies
    this with the initial rho of the network.

    This therefore assumes that the initial state of the qubits it generates
    entanglement accross is 0. It will be incorrect otherwise.
  """
  # SET UP
  num_qubits_left = q1
  num_qubits_right = (num_qubits - 1) - q2
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]


  # GENERATING ENTANGLEMENT
  # initial density matrix of bell pair, taking into account errors initial fidelity
  rho = F_initial * bell_state('phi+') + ((1 - F_initial) / 3) * (bell_state('phi-') +
                                                                       bell_state('psi+') +
                                                                       bell_state('psi-'))
  if len(qubits_left_list) != 0:
    rho_qubits_left = rho_initial.ptrace(qubits_left_list)
    rho = tensor(rho_qubits_left, rho)

  if len(qubits_right_list) != 0:
    rho_qubits_right = rho_initial.ptrace(qubits_right_list)
    rho = tensor(rho, rho_qubits_right)


  return rho

In [17]:
# testing generating entanglment twice- it works!
num_qubits = 7
rho = tensor([zero_rho for i in range (num_qubits)])

rho = entanglement_generation_with_ptracing(rho, 0, 1, 1)
rho = entanglement_generation_with_ptracing(rho, 5, 6, 1)

for i in range(num_qubits):
  print(f'Qubit{i}\n', rho.ptrace(i).full(), '\n')

Qubit0
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 

Qubit1
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 

Qubit2
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit3
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit4
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit5
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 

Qubit6
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 



In [18]:
# testing that ptracing out multiple qubits perserves entanglment- it does!
num_qubits = 2
rho = tensor(bell_state('phi+'))

qubits_list = []
print(rho.ptrace([0,1]).full())
print(tensor(rho.ptrace(0), rho.ptrace(1)).full())
print(rho.full())

[[0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]]
[[0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.25+0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.  +0.j 0.25+0.j]]
[[0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]]


In [27]:
# dephasing working
def entanglement_generation_event(rho, q1, q2):
  num_qubits_left = q1
  num_qubits_right = (num_qubits - 1) - q2

  # GENERATING ENTANGLEMENT
  # entanglement op over the 2 qubits
  entanglement_op = CNOT() * tensor(H(), I())

  # generate identity gates to the left (if there are any) and adds to the entanglement op
  entanglement_op = pad_op_to_left(entanglement_op, num_qubits_left)

  # generate identity gates to the right (if there are any) and adds to the entanglement op
  entanglement_op = pad_op_to_right(entanglement_op, num_qubits_right)

  rho = entanglement_op * rho * entanglement_op.dag()


  # ADDING MEMORY NOISE
  T_dp = 0.00001 # dephasing time

  # dephasing channel
  def dephasing_channel(rho, t, left_padding, right_padding):
    dp_prob = (1 - np.exp(-t / T_dp)) / 2
    dp_op = np.sqrt(1 - dp_prob) * qeye(2) + np.sqrt(dp_prob) * sigmaz()
    dp_op = pad_op_to_left(dp_op, left_padding)
    dp_op = pad_op_to_right(dp_op, right_padding)
    rho_t = dp_op * rho * dp_op.dag()
    return rho_t

  # dephasing channel for q1
  rho = dephasing_channel(rho, 2 * d/c, num_qubits_left, num_qubits_right + 1)

  # dephasing channel for q2
  rho = dephasing_channel(rho, d/c, num_qubits_left + 1, num_qubits_right)

  return rho


In [28]:
# testing generating entanglement w dephasing + ptracing
num_qubits = 4
rho = tensor([zero_rho for _ in range (num_qubits)])

rho = entanglement_generation_event(rho, 0, 1)

for i in range(num_qubits):
  print(f'Qubit{i}\n', rho.ptrace(i).full(), '\n')

Qubit0
 [[0.70633614+0.j 0.        +0.j]
 [0.        +0.j 0.32836473+0.j]] 

Qubit1
 [[0.70633614+0.j 0.        +0.j]
 [0.        +0.j 0.32836473+0.j]] 

Qubit2
 [[1.03470086+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 

Qubit3
 [[1.03470086+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 



In [31]:
def entanglement_generation_with_ptracing_and_dephasing(rho_initial, q1, q2, F_initial):
  """
    This function generates entanglement across 2 neighbouring repeater stations
    It does this by first creating a bell pair across 2 qubits, padding this out
    with identities so that it matches the size of the network, then it multiplies
    this with the initial rho of the network.

    This therefore assumes that the initial state of the qubits it generates
    entanglement accross is 0. It will be incorrect otherwise.
  """
  # SET UP
  num_qubits_left = q1
  num_qubits_right = (num_qubits - 1) - q2
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]


  # GENERATING ENTANGLEMENT
  # initial density matrix of bell pair, taking into account errors initial fidelity
  rho = F_initial * bell_state('phi+') + ((1 - F_initial) / 3) * (bell_state('phi-') +
                                                                       bell_state('psi+') +
                                                                       bell_state('psi-'))
  if len(qubits_left_list) != 0:
    rho_qubits_left = rho_initial.ptrace(qubits_left_list)
    rho = tensor(rho_qubits_left, rho)

  if len(qubits_right_list) != 0:
    rho_qubits_right = rho_initial.ptrace(qubits_right_list)
    rho = tensor(rho, rho_qubits_right)


  # ADDING MEMORY NOISE
  T_dp = 0.001 # dephasing time

  # dephasing channel
  def dephasing_channel(rho, t, left_padding, right_padding):
    dp_prob = (1 - np.exp(-t / T_dp)) / 2
    dp_op = np.sqrt(1 - dp_prob) * qeye(2) + np.sqrt(dp_prob) * sigmaz()
    dp_op = pad_op_to_left(dp_op, left_padding)
    dp_op = pad_op_to_right(dp_op, right_padding)
    rho_t = dp_op * rho * dp_op.dag()
    return rho_t

  # dephasing channel for q1
  rho = dephasing_channel(rho, 2 * d/c, num_qubits_left, num_qubits_right + 1)

  # dephasing channel for q2
  rho = dephasing_channel(rho, d/c, num_qubits_left + 1, num_qubits_right)

  return rho

In [35]:
# testing generating entanglment w dephasing + ptracing
num_qubits = 4
rho = tensor([zero_rho for i in range (num_qubits)])

rho = entanglement_generation_with_ptracing_and_dephasing(rho, 0, 1, 1)

for i in range(num_qubits):
  print(f'Qubit{i}\n', rho.ptrace(i).full(), '\n')

Qubit0
 [[0.51926089+0.j 0.        +0.j]
 [0.        +0.j 0.4810926 +0.j]] 

Qubit1
 [[0.51926089+0.j 0.        +0.j]
 [0.        +0.j 0.4810926 +0.j]] 

Qubit2
 [[1.00035349+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 

Qubit3
 [[1.00035349+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 



In [104]:
# testing function
def test_entang_gen_w_dens_mat(qubit1, qubit2):
  """
  Testing the entanglment generation function with statevectors
  """
  psi0 = tensor([zero_ket] * num_qubits)
  rho0 = psi0 * psi0.dag()

  for i in range (num_qubits):
    print(f'Repeater {i}\n', (entanglement_generation_event(rho0, qubit1, qubit2) * rho0 * entanglement_generation_event(rho0, qubit1, qubit2).dag()).ptrace(i).full(), '\n')
test_entang_gen_w_dens_mat(0,1)


Repeater 0
 [[0.31591148+0.j 0.        +0.j]
 [0.        +0.j 0.24813201+0.j]] 

Repeater 1
 [[0.31591148+0.j 0.        +0.j]
 [0.        +0.j 0.24813201+0.j]] 

Repeater 2
 [[0.56404349+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 

Repeater 3
 [[0.56404349+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 

Repeater 4
 [[0.56404349+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 

